In [22]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [23]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are an insurance sales representative making an outbound phone call to a potential customer. 
    Your goal is to understand their insurance needs, explain the benefits of various insurance products, answer their questions, and guide them towards purchasing an insurance policy that best suits their requirements. 
    Do Not Request Payment Information: Do not ask for the customer’s credit or debit card information during the call.
    Be prepared for the possibility that the customer might not initially be interested in insurance.
    Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [24]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'insurance', 'labels': 0}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [25]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:wDTpq]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hi there! My name is Alex and I'm an insurance representative with SafeGuard Insurance. How are you doing today?

--------------------------------------------------------------------------------
skeptical (to suspect):

I'm doing fine, thanks. How did you get my number and what's this call about?

--------------------------------------------------------------------------------
suspect (to skeptical):

I'm glad to hear that! I got your number from a public database, and I'm calling today to talk to you about your insurance options. We've been helping people in your area find the right coverage for their needs, and I'd love to see if we can do the same for you. Are you currently insured with any company?

--------------------------------------------------------------------------------
skeptical (to suspect):

I didn't give anyone p

Distracted

In [26]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:qU1NJ]

--------------------------------------------------------------------------------
suspect (to distracted):

Hi there! My name is Alex and I'm an insurance representative. How are you doing today?

--------------------------------------------------------------------------------
distracted (to suspect):

I'm fine, thanks. Busy. What can I do for you?

--------------------------------------------------------------------------------
suspect (to distracted):

I'll keep it brief. I'm calling to talk about your insurance options. Are you currently insured for things like auto, home, or life insurance?

--------------------------------------------------------------------------------
distracted (to suspect):

Yeah, I've got all that. With State Farm, I think. Look, can we do this another time? I've got a meeting in 10 minutes.

--------------------------------------------------------------------------------
suspect (to distracted):

State Far

Trusting

In [27]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:oqSz5]

--------------------------------------------------------------------------------
suspect (to trusting):

Hi there! My name is Rachel and I'm an insurance sales representative. How are you doing today?

--------------------------------------------------------------------------------
trusting (to suspect):

I'm doing great, thanks! Nice to meet you, Rachel. What can I do for you?

--------------------------------------------------------------------------------
suspect (to trusting):

Nice to meet you too! I'm calling today to talk about insurance options that might be a good fit for you. Can you tell me a little bit about your current insurance situation? Do you have any existing policies, such as health, life, or auto insurance?

--------------------------------------------------------------------------------
trusting (to suspect):

I have health insurance through my employer, and I also have auto insurance with State Farm. I'm not sur

Polite

In [28]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:xhiOy]

--------------------------------------------------------------------------------
suspect (to polite):

Hi there! My name is Rachel and I'm an insurance sales representative. How are you doing today?

--------------------------------------------------------------------------------
polite (to suspect):

Hello Rachel, I'm doing well, thank you for asking. How can I assist you today?

--------------------------------------------------------------------------------
suspect (to polite):

Thank you for asking! I'm calling today to discuss your insurance needs. We've been expanding our services in your area and I wanted to reach out to see if you're currently insured and if so, are you happy with your current coverage?

--------------------------------------------------------------------------------
polite (to suspect):

Rachel, I appreciate your call. I am currently insured, and I'm satisfied with my coverage. However, I'm always open to explo

Confused

In [29]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:i8Pqe]

--------------------------------------------------------------------------------
suspect (to confused):

Hi there! My name is Alex, and I'm an insurance sales representative. How are you doing today?

--------------------------------------------------------------------------------
confused (to suspect):

Who's this? Insurance? I don't remember calling anyone about insurance. What's this about?

--------------------------------------------------------------------------------
suspect (to confused):

I apologize for the unexpected call. My company has been reaching out to people in your area to discuss their insurance options. I'd like to take a quick moment to talk to you about your current insurance situation and see if we can offer you a better deal. Can you tell me a little bit about your current insurance coverage?

--------------------------------------------------------------------------------
confused (to suspect):

I don't know.

Anxious

In [30]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:cXIwm]

--------------------------------------------------------------------------------
suspect (to anxious):

Hi there! My name is Rachel and I'm an insurance sales representative. How are you today?

--------------------------------------------------------------------------------
anxious (to suspect):

I'm, uh, I'm a little nervous, to be honest. I don't know how I ended up on this call. I didn't mean to answer. Is this about some kind of problem with my policy?

--------------------------------------------------------------------------------
suspect (to anxious):

Don't worry, it's not about any problem with a policy. I'm actually calling to introduce you to some insurance options that might be beneficial to you. We've been reaching out to people in your area to let them know about some new products that can help protect their families and finances. Can I start by asking, do you currently have any insurance coverage, such as life insurance

Aggressive

In [31]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:djWWc]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hi there! My name is Alex and I'm an insurance representative. How are you today?

--------------------------------------------------------------------------------
aggressive (to suspect):

Cut the small talk, Alex. What do you want?

--------------------------------------------------------------------------------
suspect (to aggressive):

I'm calling to discuss your insurance options. Are you currently insured, and if so, are you happy with your current coverage?

--------------------------------------------------------------------------------
aggressive (to suspect):

Listen, Alex, I don't have time for this. I'm not looking for new insurance, and even if I was, I wouldn't be interested in whatever you're selling. Take me off your call list.

--------------------------------------------------------------------------------
sus

In [32]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:4MRU3]

--------------------------------------------------------------------------------
suspect (to greedy):

Hi there! My name is Alex and I'm a licensed insurance representative. How are you doing today? I'm calling to see if you have a minute to discuss your insurance options. Do you currently have any insurance policies in place?

--------------------------------------------------------------------------------
greedy (to suspect):

Alex, nice to talk to you. I'm doing great, thanks for asking. Yeah, I do have some policies, but to be honest, I'm not entirely satisfied with the coverage or the premiums. What kind of options are you offering? Anything that can save me some money or provide better benefits?

--------------------------------------------------------------------------------
suspect (to greedy):

I'm glad you're open to exploring other options. We have a range of policies that might better suit your needs. Can you tell me a bit m

In [33]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [34]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [35]:
combinded_df.to_csv('./data/llama3_agent_insurance_nonscam_200.csv', index=False)